## Making MDPs in Python

For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

You can start this assignment during/after reading Grokking Ch-2.

NOTE: all page numbers referred to in this ipynb notebook refer to Grokking textbook

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [3]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Here we a dictionary bw_mdp which to get the transitions possible we can do bw_mdp\[current state\]\[action to take\]. this will return a list of the possibilities.

The format of each element within the list is   
(probability of that transition , next state , reward , 'is the next state a terminal state?')

Obviously make sure that the sum of probability of all possible transitions is 1

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

And similiarly vice versa for Left(just replace left with right and right with left in previous list)

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [5]:
swf_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    1 : {
        "Right" : [
            (1/2, 2, 0, False),
            (1/3, 1, 0, False),
            (1/6, 0, 0, True),
        ],
        "Left" : [
            (1/2, 0, 1, True),
            (1/3, 1, 0, False),
            (1/6, 2, 0, False),
        ]
    }, 

    # to be added 
    2 : {
        "Right" : [
            (1/2, 3, 0, False),
            (1/3, 2, 0, False),
            (1/6, 1, 0, False),
        ],
        "Left" : [
            (1/2, 1, 0, False),
            (1/3, 2, 0, False),
            (1/6, 3, 0, False),
        ]
    },
    3 :{
        "Right" : [
            (1/2, 4, 0, False),
            (1/3, 3, 0, False),
            (1/6, 2, 0, False),
        ],
        "Left" : [
            (1/2, 2, 0, False),
            (1/3, 3, 0, False),
            (1/6, 4, 0, False),
        ]
    },
    4 : {
        "Right" : [
            (1/2, 5, 0, False),
            (1/3, 4, 0, False),
            (1/6, 3, 0, False),
        ],
        "Left" : [
            (1/2, 3, 0, False),
            (1/3, 4, 0, False),
            (1/6, 5, 0, False),
        ]
    },
    5 : {
        "Right" : [
            (1/2, 6, 1, True),
            (1/3, 5, 0, False),
            (1/6, 4, 0, False),
        ],
        "Left" : [
            (1/2, 4, 0, False),
            (1/3, 5, 0, False),
            (1/6, 6, 0, True),
        ]
    },
    6 : {
        "Right" : [(1, 6, 0, True)],
        "Left" : [(1, 6, 0, True)],
    },
    
}

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [10]:
fl_mdp = {}
def get_next_state(state, action):

    row = state//4
    col = state%4

    if action == "Up":
        row = max(0, row - 1)
    elif action == "Down":
        row = min(3, row + 1)
    elif action == "Left":
        col = max(0, col - 1)
    elif action == "Right":
        col = min(3, col + 1)

    return row * 4 + col



In [8]:
terminal_states = {5, 7, 11, 12, 15}

for state in range(16):
    transitions = {}

    if state in terminal_states:

        for action in ["Up", "Down", "Right", "Left"]:
            reward = 1 if state == 15 else 0
            transitions[action] = [(1.0, state, reward, True)]
    else:

        actions = {
            "Up": ["Up", "Left", "Right"],
            "Down": ["Down", "Left", "Right"],
            "Left": ["Left", "Up", "Down"],
            "Right": ["Right", "Up", "Down"]
        }

        for action in ["Up", "Down", "Right", "Left"]:
            k = actions[action]
            transition_list = []

            for i in k:
                next_state = get_next_state(state, i)
                terminal = next_state in terminal_states
                reward = 1 if next_state == 15 else 0
                transition_list.append((1/3, next_state, reward, terminal))

            transitions[action] = transition_list

    fl_mdp[state] = transitions

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.
OpenAI Gym basically contains implmentations of common RL MDPs so that we can directly train our agents on test environments. 

In [4]:
import gym
P = gym.make('FrozenLake-v1').env.P

Since the imported MDP is also just a dictionary, we can just print it.

In [5]:
# using the pretty print module

import pprint
pprint.pprint(P)

{0: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     2: [(0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     3: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False)],
     2: [(0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     3: [(0.3333333333333333,

## Environment 3 - Key Door Game

Now we will implement a slightly different version of frozen lake without the stochasticity but with the added constraint of a key.

So now we have a 4x4 grid in which some cells are holes(and so terminal) and we have a goal state(keep it as 15) but this time someone locked the goal state with a door :(. Without the key we CANNOT ENTER the goal state.

Luckily for us, that guy was very clumsy and dropped the key in tile 3!. So now make a MDP to represent the process of us escaping this grid world.

Remember that the state in a MDP isnt just the position/location we are in. it should represent our whole game situation. (For example I can either be at tile 14 with the key or at tile 14 without key and these two situations are different and therfore must be represented as **different** states!)

To summarise
- state state is 0
- door/goal state is 15
- holes at 5, 7, 11, 12
- key at 3
- The key is picked up automatically upon entering tile 3
- can only enter tile 15 when key is obtained(otherwise its like a wall)
- holes & goal are terminal
- reward of 1 when reaching tile 15 wiht the key
- Attempting to enter tile 15 when we DONT have the key should be treated like hitting a wall and bouncing back to the original state where we took that action

In [13]:
gw_map = {}

# I am using bool key in dictionary to represent presence of key

def get_next_pos(pos, action):

    row = pos/4
    col = pos%4

    if action == "Up":
        row = max(0, row - 1)
    elif action == "Down":
        row = min(3, row + 1)
    elif action == "Left":
        col = max(0, col - 1)
    elif action == "Right":
        col = min(3, col + 1)

    return row * 4 + col

holes = {5, 7, 11, 12}
key_pos = 3
goal = 15

for pos in range(0,16):
    gw_map[pos] = {}


    for key in [False, True]:  #For both cases if he has key or not
        transitions = {}


        if pos in holes or (pos == goal and key):         #if action is given in goal or hole
            for action in ["Up", "Down", "Left", "Right"]:
                reward = 1 if (pos == goal ) else 0
                transitions[action] = [(1.0, pos, key, reward, True)]


        else:
            for action in ["Up", "Down", "Left", "Right"]:
                next_pos = get_next_pos(pos, action)

                if next_pos == goal and not key:        #no change in position if he doesn't have key
                    next_pos = pos

                if next_pos == key_pos:    #If key is found
                    key = True

                terminal = next_pos in holes or (next_pos == goal and key)
                reward = 1 if (next_pos == goal and key) else 0

                transitions[action] = [(1.0, next_pos, key, reward, terminal)]

        gw_map[pos][key] = transitions


The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: For additional challenges
- Make this grid-world also stochastic like Frozen Lake
- Make a function that allows you to input in the locations of the holes, the door(goal state) and the key and using that it will generate and return the MDP

## Environment 4 - Lights Out

**Lights out** is a deterministic puzzle game which consists of a 5x5 grid of lights which can either be on or off.

When the game begins a random set of these lights will be switched on. Pressing any light will toggle it (toggle means if the light was on then it will become off and if it was off then it will become on) and it will also toggle all of its neighbours.

![image of the game](./images/LightsOutIllustration.png)

Note: so any non-edge and non-corner cell action will toggle 5 lights, an edge cell action would toggle 4 lights and a corner cell action only toggles 3 lights.


The goal of the game is to find a sequence of moves to switch off all the lights. Now i'm not gonna make you solve this instead we are going to represent this as a deterministic MDP, where 
- the states should represent the game grid layout 
- the actions are the cells you wish to toggle
- Give a reward of 1 when solved 
- the ONLY terminal state is the fully solved state

Since 5x5 is pretty big we will restrict ourselves to a **4x4** grid and represent it as an MDP instead

In [ ]:
#Representing the State in form of 16 bits like 0010011010010111 where 1 represent on and 0 as off
#first tried using 2d arrays but i didn't know about tuples and mutable etc. clearly so AI suggested to do bit manipulation

def toggle_lights(state, action):

    row = action // 4
    col = action % 4
    new_state = state

    # Toggle the pressed light
    new_state ^= (1 << action)

    # Toggle neighbors (up, down, left, right)
    if row > 0:  # Up
        new_state ^= (1 << (action - 4))
    if row < 3:  # Down
        new_state ^= (1 << (action + 4))
    if col > 0:  # Left
        new_state ^= (1 << (action - 1))
    if col < 3:  # Right
        new_state ^= (1 << (action + 1))

    return new_state

lo_map = {}

for state in range(2**16):
    transitions = {}

    if state == 0:  # All lights off
        for action in range(16):
            next_state = toggle_lights(state, action)
            transitions[action] = [(1.0, next_state, 1, False)] #since it was solved so reward is 1
    else:
        for action in range(16):
            next_state = toggle_lights(state, action)
            is_terminal = (next_state == 0)
            reward = 1 if is_terminal else 0
            transitions[action] = [(1.0, next_state, reward, is_terminal)]

    lo_map[state] = transitions

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: if you want you can also try to represent the 5x5 one but this is optional